<a href="https://colab.research.google.com/github/ykitaguchi77/CorneAI/blob/main/YOLOv5_CutMix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**YOLOv5_CutMix**

In [1]:
# prompt: gdriveをマウント

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [11]:
folder_path = '/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の240問_CutMix/CutMix_images'